In [1]:
%%timeit

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pywt
import glob
import pandas as pd
from pandas.compat import StringIO
from scipy import signal
from scipy import stats
from scipy.signal import sosfiltfilt, butter
import time

def window_stack(a, stepsize=1, width=3):
    n = int(np.floor((a.shape[1] - width) / stepsize))
    return np.dstack( a[:,i*stepsize:width+i*stepsize] for i in range(0,n) )

def minmax_norm(x):
    return 2*(x - np.min(x))/(np.max(x) - np.min(x)) - 1
            
def event_log_analyzer(log_file_path=None, output_form=None, fs=512):
    
    event_log = pd.read_csv(event_f_name, delimiter = '\t', error_bad_lines=False, warn_bad_lines=False, 
                        encoding = 'ISO-8859-1', skiprows=10, parse_dates = ['Name', 'Time', 'Duration'])
    
    event_list = list(event_log.Name)
    time_list = list(event_log.Time)
    duration_list = list(event_log.Duration)

    indices = np.array([i for i, x in enumerate(event_list) if 'Seizure' in x])

    sz_event_list = []

    for i in indices:
        time_split = str(time_list[i]).split()
        sz_onset_hr = int(time_split[1].split(':')[0])
        sz_onset_min = int(time_split[1].split(':')[1])
        sz_onset_sec = int(time_split[1].split(':')[2])
    
        dur_split = str(duration_list[i]).split()

        sz_dur_min = int(dur_split[-1].split(':')[0])
        sz_dur_sec = int(dur_split[-1].split(':')[1])
   
        sz_off_hr = sz_onset_hr
        sz_off_min = sz_onset_min + sz_dur_min
    
        if sz_off_min >= 60:
            sz_off_hr += 1
            sz_off_min -= 60
    
        sz_off_sec = sz_onset_sec + sz_dur_sec
    
        if sz_off_sec >= 60:
            sz_off_min += 1
            sz_off_sec -= 60
    
        if output_form == 'sec':
            sz_onset_time = 3600 * sz_onset_hr + 60 * sz_onset_min + sz_onset_sec
            sz_off_time = 3600 * sz_off_hr + 60 * sz_off_min + sz_off_sec
        elif output_form == 'sample':
            sz_onset_time = (3600 * sz_onset_hr + 60 * sz_onset_min + sz_onset_sec) * fs
            sz_off_time = (3600 * sz_off_hr + 60 * sz_off_min + sz_off_sec) * fs
        elif output_form == 'slice':
            sz_onset_time = (3600 * sz_onset_hr + 60 * sz_onset_min + sz_onset_sec) * fs / 128
            sz_off_time = (3600 * sz_off_hr + 60 * sz_off_min + sz_off_sec) * fs / 128
        else:
            sz_onset_time = '{:02d}:{:02d}:{:02d}'.format(sz_onset_hr,sz_onset_min,sz_onset_sec)
            sz_off_time = '{:02d}:{:02d}:{:02d}'.format(sz_off_hr,sz_off_min,sz_off_sec)
        
        sz_event_list.append([sz_onset_time, sz_off_time])
        
    return sz_event_list            

# Parameters for spectrogram
fs = 512
n_fft = 512
n_perseg = 128
# Data window length
out_shape = 128
n_step = 64

# Training File location
file_list = sorted(glob.glob('/home/sungmin2/py35_codes/00_AnoGAN/EEG_DATA/test4coderuntime/sz/*.txt'))

# Data output path
output_path = '/home/sungmin2/py35_codes/00_AnoGAN/EEG_DATA/test4coderuntime/sz/processed/'
os.makedirs(output_path, exist_ok=True) 

# get the start time
st = time.process_time()

for file_name in file_list:
#     print('*'*5 + ' processing: ' + file_name)
    fname_split = file_name.split(".")
    if "event_log" in fname_split[0]: 
        continue
    fname_split_id = fname_split[0].split("/")
    fname_id = fname_split_id[-1]

    event_f_name = fname_split[0] + '_event_log.txt'

    sz_log = event_log_analyzer(event_f_name, output_form='sec')
    data_read = pd.read_csv(file_name, delimiter = '\t', chunksize=fs * 20, usecols = ['T9','T10', 'TP9', 'TP10'], 
                        na_values=['T9','T10', 'TP9', 'TP10'], dtype=np.float64 ).read()
    
    T9_signal_data = np.array(data_read['T9'])
    T9_signal_data = T9_signal_data[~np.isnan(T9_signal_data)]

    T10_signal_data = np.array(data_read['T10'])
    T10_signal_data = T10_signal_data[~np.isnan(T10_signal_data)]
    
    TP9_signal_data = np.array(data_read['TP9'])
    TP9_signal_data = TP9_signal_data[~np.isnan(TP9_signal_data)]

    TP10_signal_data = np.array(data_read['TP10'])
    TP10_signal_data = TP10_signal_data[~np.isnan(TP10_signal_data)]
    
    T_signal_data = T9_signal_data - T10_signal_data
    TP_signal_data = TP9_signal_data - TP10_signal_data

    sos = butter(16, np.array([0.5 , 120]) / (0.5*fs) , 'bandpass', output='sos')

    sz_off = 0
    time_idx = 0
    
    if sz_log:
        
        T_signal_filtered = sosfiltfilt(sos, T_signal_data)
        # Spectrogram
        f, t, Sxx = signal.spectrogram(T_signal_filtered, fs, nperseg = n_perseg, noverlap = 0.5*n_perseg, 
                                           nfft=n_fft, scaling = 'density', mode='psd')
        #주파수 범위 자르기
        Sxx_n = Sxx[0:out_shape,:]        
        # Clip Z-score normalization
        T_norm_Sxx = minmax_norm(stats.zscore(Sxx_n))
        
        TP_signal_filtered = sosfiltfilt(sos, TP_signal_data)
        # Spectrogram
        f, t, Sxx = signal.spectrogram(TP_signal_filtered, fs, nperseg = n_perseg, noverlap = 0.5*n_perseg, 
                                           nfft=n_fft, scaling = 'density', mode='psd')
        #주파수 범위 자르기
        Sxx_n = Sxx[0:out_shape,:]
        # Clip Z-score normalization
        TP_norm_Sxx = minmax_norm(stats.zscore(Sxx_n))
        
        for event_idx, event in enumerate(sz_log):

            onset_point = np.where(t==event[0])[0]
            offset_point = np.where(t==event[1])[0]
            
            T_Sxx_non_ictal = T_norm_Sxx[:,sz_off:onset_point[0]]
            T_Sxx = np.moveaxis(window_stack(T_Sxx_non_ictal, stepsize=n_step, width = out_shape),2,0)

            TP_Sxx_non_ictal = TP_norm_Sxx[:,sz_off:onset_point[0]]
            TP_Sxx = np.moveaxis(window_stack(TP_Sxx_non_ictal, stepsize=n_step, width = out_shape),2,0)
    
            X = np.stack((T_Sxx,TP_Sxx), axis = 3)
        
            X_out= np.concatenate([X, np.expand_dims(np.mean(X, axis = 3), axis = 3)], axis = 3)
#             print('Norm interval shape : ' + str(X_out.shape))
            
            for slice_idx in range(0, X_out.shape[0]):
                np.save(output_path+ fname_id+'-{:06d}_norm'.format(time_idx),X_out[slice_idx,:,:,:])
                time_idx += 1

            T_norm_Sxx_ictal = T_norm_Sxx[:,(onset_point[0]):(offset_point[0])]
            T_Sxx = np.moveaxis(window_stack(T_norm_Sxx_ictal, stepsize=n_step, width = out_shape),2,0)
            
            TP_norm_Sxx_ictal = TP_norm_Sxx[:,(onset_point[0]):(offset_point[0])]
            TP_Sxx = np.moveaxis(window_stack(TP_norm_Sxx_ictal, stepsize=n_step, width = out_shape),2,0)
            
            X = np.stack((T_Sxx,TP_Sxx), axis = 3)
            
            X_out= np.concatenate([X, np.expand_dims(np.mean(X, axis = 3), axis = 3)], axis = 3)
#             print('Ictal interval shape : ' + str(X_out.shape))
            
            sz_off = offset_point[0]
            
            for slice_idx in range(0, X_out.shape[0]):
                np.save(output_path+ fname_id+'-{:06d}_ictal'.format(time_idx),X_out[slice_idx,:,:,:])
                time_idx += 1
                                
#             if (event[1] > 290) :
#                 continue
                
#                 T_Sxx_non_ictal = T_norm_Sxx[:,sz_off:]
#                 T_Sxx = np.moveaxis(window_stack(T_Sxx_non_ictal, stepsize=n_step, width = out_shape),2,0)

#                 TP_Sxx_non_ictal = TP_norm_Sxx[:,sz_off:]
#                 TP_Sxx = np.moveaxis(window_stack(TP_Sxx_non_ictal, stepsize=n_step, width = out_shape),2,0)

#                 X = np.stack((T_Sxx,TP_Sxx), axis = 3)
#                 X_out= np.concatenate([X, np.expand_dims(np.mean(X, axis = 3), axis = 3)], axis = 3)
#                 sz_off = offset_point[0]
#                 print('Norm interval shape : ' + str(X_out.shape))

#                 for slice_idx in range(0, X_out.shape[0]):
#                     np.save(output_path+ fname_id+'-{:06d}_norm'.format(time_idx),X_out[slice_idx,:,:,:])
#                     time_idx += 1

                
#             if (event_idx == (len(sz_log) - 1)):
#                 T_Sxx_non_ictal = T_norm_Sxx[:,sz_off:]
#                 T_Sxx = np.moveaxis(window_stack(T_Sxx_non_ictal, stepsize=n_step, width = out_shape),2,0)

#                 TP_Sxx_non_ictal = TP_norm_Sxx[:,sz_off:]
#                 TP_Sxx = np.moveaxis(window_stack(TP_Sxx_non_ictal, stepsize=n_step, width = out_shape),2,0)
    
#                 X = np.stack((T_Sxx,TP_Sxx), axis = 3)
#                 X_out= np.concatenate([X, np.expand_dims(np.mean(X, axis = 3), axis = 3)], axis = 3)
#                 sz_off = offset_point[0]
#                 print('Norm interval shape : ' + str(X_out.shape))
                
#                 for slice_idx in range(0, X_out.shape[0]):
#                     np.save(output_path+ fname_id+'-{:06d}_norm'.format(time_idx),X_out[slice_idx,:,:,:])
#                     time_idx += 1

# get the end time
et = time.process_time()

# get execution time
res = et - st
final_res = res * 1000

print('CPU Execution time:', final_res, 'milliseconds')

# print('*'*20 + 'DONE' + '*'*20)

CPU Execution time: 579.568804 milliseconds
CPU Execution time: 704.9943129999994 milliseconds
CPU Execution time: 633.223986 milliseconds
CPU Execution time: 656.2421760000001 milliseconds
CPU Execution time: 562.0180159999996 milliseconds
CPU Execution time: 605.8878200000004 milliseconds
CPU Execution time: 542.0591860000012 milliseconds
CPU Execution time: 602.1114759999992 milliseconds
633 ms ± 77.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
